## Notebook de Treinamento dos Modelos

In [4]:
#Origem dos dados
import yfinance as yf

In [5]:
# Data manipulation
import pandas as pd
import numpy as np
from datetime import date
from datetime import timedelta

In [6]:
# Plots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['font.size'] = 10

In [7]:
#Gráfico padrão

def cria_grafico( df1, df2, df3, df4, titulo, fixa_data, len_fixa_data):
    if fixa_data ==1 :
      dt_ini = df1.index[-(len_fixa_data):]
    else:
      dt_ini = df1.index[:1]

    dt_fim = df1.index[-1:]

    layout = go.Layout(
                    title = titulo,
                    titlefont = dict(size=20))
    fig = go.Figure(layout=layout)
    fig.add_trace(go.Scatter(x=df1.index.values, y=df1[df1.columns[0]].values,
                    mode='lines',
                    line_color = 'blue',
                    name=df1.columns[0]))

    if df2.shape[0] != 0 :
      fig.add_trace(go.Scatter(x=df2.index.values, y=df2[df2.columns[0]].values,
                    mode='lines',
                    line_color = 'red',
                    name=df2.columns[0]))

    if df3.shape[0] != 0 :
      fig.add_trace(go.Scatter(x=df3.index.values, y=df3[df3.columns[0]].values,
                    mode='lines',
                    line_color = 'green',
                    name=df3.columns[0]))

    if df4.shape[0] != 0 :
      fig.add_trace(go.Scatter(x=df4.index.values, y=df4[df4.columns[0]].values,
                    mode='lines',
                    line_color = 'orange',
                    name=df4.columns[0]))

    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
                        ])
            ),
        range=(dt_ini[0].strftime('%Y-%m-%d'), dt_fim[0].strftime('%Y-%m-%d'))
    )
    return fig

In [8]:
#Fonte dos índices e cotações:   https://finance.yahoo.com/
indice = "BZ=F"   # Brent Crude Oil Last Day Financ (BZ=F)
inicio = "2009-01-01" #Define a data de ínicio para importação dos dados
#Coleta dados históricos do índice de referência até a data corrente
dados_acao = yf.download(indice, inicio) #Quando a biblioteca é chamada sem uma data final, carrega as cotações até a data corrente
df_cotacoes = pd.DataFrame({indice: dados_acao['Close']})

[*********************100%%**********************]  1 of 1 completed


Comportamento gráfico das cotações do índice no período:

In [9]:
df1 = pd.DataFrame(df_cotacoes)
df1.rename(columns={indice: 'Brent Crude Oil'}, inplace = True)
df2 = pd.DataFrame(data=[])
df3 = pd.DataFrame(data=[])
df4 = pd.DataFrame(data=[])

fig = cria_grafico(df1, df2,df3,df4,'Histórico de Cotações Brent Crude Oil', 0,0 )
fig.show()

Análises de Oscilações

In [10]:
mean_value = df_cotacoes[indice].describe().loc[['mean']].mean()
media_valor_brent = round(float(mean_value), 2)
print("No gráfico acima, podemos verificar algumas oscilações bruscas na cotação do Brent Crude Oil ao longo dos últimos 15 anos. Na média, as cotações ficaram na casa dos USD " + str(media_valor_brent) + ", sendo que em abril/2020 a cotação atingiu seu menor preço no período, ficando em USD 19,33 e em março/2022 atingiu o maior valor do período, ficando em USD 127,98. Abaixo vamos detalhar um pouco esta variações")
pd.options.display.float_format = "{:.2f}".format
df_cotacoes[indice].describe().loc[['mean','min','max']]

No gráfico acima, podemos verificar algumas oscilações bruscas na cotação do Brent Crude Oil ao longo dos últimos 15 anos. Na média, as cotações ficaram na casa dos USD 77.26, sendo que em abril/2020 a cotação atingiu seu menor preço no período, ficando em USD 19,33 e em março/2022 atingiu o maior valor do período, ficando em USD 127,98. Abaixo vamos detalhar um pouco esta variações


mean    77.26
min     19.33
max    127.98
Name: BZ=F, dtype: float64

Modelos e Validadores

In [11]:
#Importações
from statsmodels.tsa.stattools import adfuller # importar o teste ADF
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error,  mean_absolute_percentage_error, r2_score
import xgboost as xgb
from xgboost import XGBRegressor


In [12]:
#Define Constantes para todos os  Modelo
steps = 120  #Tamanho da base de testes. Optamos por treinar com todo o histórico e testar com ultimos x dias definidos na variável
du = 10       #dias úteis para previsão futura
Lista_indicadores = ['Erro Médio Absoluto - MAE','Erro Quadrático Médio - MSE','Raiz Quadrada do Erro Médio - RMSE','Média Percentual Absoluta do Erro - MAPE','Coeficiente de Determinação(R²)']


MODELO ARIMA

O modelo Arima (AutoRegressive Integrated Moving Average) é amplamente utilizado para previsão de séries temporais. Ele combina componentes de regressão autoregressiva, média móvel e diferenciação para capturar os padrões presentes nos dados. É mandatório que a série temporal seja estacionária para que este modelo seja aplicado.

In [14]:
#Define um valor para as janelas
janela = 7

#Cria um novo DataFrame para armazenar apenas a Data e a cotação de fechamento do índice (Close)
df_arima = pd.DataFrame({indice: dados_acao['Close']})
df_arima = df_arima.reset_index('Date')
df_arima['Date'] = pd.to_datetime(df_arima['Date']) #realizando a conversão da data para formato datetime
df_arima.set_index('Date', inplace = True)

X = df_arima[indice].values

# aplicar ADF e imprimir o resultado
result = adfuller(X)
ma = df_arima.rolling(janela).mean()
df_log = np.log(df_arima)
ma_log = df_log.rolling(janela).mean()

#subtrair média do log dos dados
df_sub = (df_log - ma_log).dropna()
ma_sub = df_sub.rolling(janela).mean()

#desvio padrão
std_sub = df_sub.rolling(janela).std()

#repetir o ADF
X_sub = df_sub[indice].values

# aplicar ADF
result_sub = adfuller(X_sub)

#Diferenciação
#aplicar diferenciação
df_diff = df_sub.diff(1)
ma_diff = df_diff.rolling(janela).mean()

#desvio padrão
std_diff = df_diff.rolling(janela).std()

#extrair apenas os valores e retirar os valores NA
X = df_diff[indice].dropna().values

# aplicar ADF e imprimir o resultado
result_diff = adfuller(X)

lag_acf = acf(df_diff.dropna(), nlags=janela)
lag_pacf = pacf(df_diff.dropna(), nlags=janela)

#Treinando o modelo
#ARIMA(p,d,q)
p = 1
d = 1
q = 1

#Treina o modelo
model_arima = ARIMA(df_log, order=(p,d,q))
result_AR = model_arima.fit( )

#Cria um dataframe para acompanhar os ultimos x dias da base de teste com as predições
df_teste_arima  = pd.DataFrame(data=df_log[-steps:], columns=[indice])
df_i = df_teste_arima.head(1)
df_i.reset_index(inplace = True)
di = df_i['Date'].astype(str)

df_e = df_teste_arima.tail(1)
df_e.reset_index(inplace = True)
de = df_e['Date'].astype(str)

#Cria um dataframes com os ultimos x dias da previsão do Arima
forecast =  result_AR.predict(start=di.values[0], end=de.values[0])
df_forecast =  pd.DataFrame(data= forecast.values, columns=[indice], index = df_teste_arima.index)

# Plota um gráfico para comparar o realizado com o periodo testado + previsões
df1 = pd.DataFrame(df_log)
df1.rename(columns={indice: 'Dados Históricos'}, inplace = True)
df2 = pd.DataFrame(df_forecast)
df2.rename(columns={indice: 'Predições'}, inplace = True)
df3 = pd.DataFrame(data=[])
df4 = pd.DataFrame(data=[])

figArima = cria_grafico(df1, df2,df3,df4,'Predições com Modelo Arima', 1,steps+1 )
fig.show()

#Retorna o df do ARIMA para os valores antes da diferenciação para a plotagem no gráfico comparativo
df_original= df_arima[-steps:]
cols = df_original.columns
x = []
for col in cols:
    diff_results = df_original[col] + df_forecast[col].shift(-1)
    x.append(diff_results)
diff_df_inverted = pd.concat(x, axis=1)

#calcula as métricas de avaliação de desempenho do modelo
MAE_ARIMA = mean_absolute_error(df_teste_arima[indice].values, df_forecast[indice].values)
MSE_ARIMA = mean_squared_error(df_teste_arima[indice].values, df_forecast[indice].values, squared=True)
RMSE_ARIMA = mean_squared_error(df_teste_arima[indice].values, df_forecast[indice].values, squared=False)
MAPE_ARIMA = mean_absolute_percentage_error(df_teste_arima[indice].values, df_forecast[indice].values)
r2_ARIMA = r2_score(df_teste_arima[indice].values, df_forecast[indice].values)
dados_arima = {
'Indicador': Lista_indicadores,
'Resultado': [MAE_ARIMA, MSE_ARIMA,RMSE_ARIMA,MAPE_ARIMA,r2_ARIMA]
}
df_result_arima = pd.DataFrame(data = dados_arima['Resultado'], index=dados_arima['Indicador'], columns =['Resultado'])
df_result_arima

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



,Resultado
Erro Médio Absoluto - MAE,0.01
Erro Quadrático Médio - MSE,0.00
Raiz Quadrada do Erro Médio - RMSE,0.02
Média Percentual Absoluta do Erro - MAPE,0.00
Coeficiente de Determinação(R²),0.91


MODELO PROPHET

Prophet é uma biblioteca de predição da Meta(Facebook), que segue o modelo da API sklearn.
    

In [15]:
#Cria um novo DataFrame para armazenar apenas a Data e a cotação de fechamento do índice (Close
df_prophet = pd.DataFrame({indice: dados_acao['Close']})

df_prophet = df_prophet.reset_index('Date')
df_prophet[['ds','y']] = df_prophet[['Date',indice]]

#separando os dados em Treinamento e Teste
train_pp = df_prophet.sample(frac=0.8, random_state=0)
train_pp.drop(['Date',indice], axis=1, inplace = True)
test_pp = df_prophet.drop(train_pp.index)

#Treinando o modelo
modelo_pp = Prophet(daily_seasonality='auto')
modelo_pp.fit(df_prophet)
dataFramefuture_pp = modelo_pp.make_future_dataframe(periods=du)
previsao_pp = modelo_pp.predict(dataFramefuture_pp)

# Extrair as colunas relevantes dos DataFrames
previsao_cols = ['ds', 'yhat']
valores_reais_cols = ['ds', 'y']

previsao_pp = previsao_pp[previsao_cols]
valores_reais = train_pp[valores_reais_cols]

# Mesclar os DataFrames nas colunas 'ds' para comparar previsões e valores reais
resultados_pp = pd.merge(previsao_pp, valores_reais, on='ds', how='inner')

# Plota um gráfico para comparar o realizado com o periodo testado + previsões
df1 = pd.DataFrame(data= resultados_pp['y'].values, index=resultados_pp['ds'], columns=['Dados Históricos'] )
df1.rename(columns={indice: 'Dados Históricos'}, inplace = True)
df2 = pd.DataFrame(data= resultados_pp[-steps:]['yhat'].values, index=resultados_pp[-steps:]['ds'].values, columns=['Predições'] )
df2.rename(columns={indice: 'Predições'}, inplace = True)
df3 = pd.DataFrame(data=[])
df4 = pd.DataFrame(data=[])

figProphet = cria_grafico(df1, df2,df3,df4,'Predições com Modelo Prophet', 1,steps+1 )
fig.show()

#calcula as métricas de avaliação de desempenho do modelo
MAE_pp = mean_absolute_error(resultados_pp[-steps:]['y'].values, resultados_pp[-steps:]['yhat'].values)
MSE_pp = mean_squared_error(resultados_pp[-steps:]['y'].values, resultados_pp[-steps:]['yhat'].values, squared=True)
RMSE_pp = mean_squared_error(resultados_pp[-steps:]['y'].values, resultados_pp[-steps:]['yhat'].values, squared=False)
MAPE_pp = mean_absolute_percentage_error(resultados_pp[-steps:]['y'].values, resultados_pp[-steps:]['yhat'].values)
r2_pp = r2_score(resultados_pp[-steps:]['y'].values, resultados_pp[-steps:]['yhat'].values)
dados_prophet = {
'Indicador': Lista_indicadores,
'Resultado': [MAE_pp, MSE_pp,RMSE_pp,MAPE_pp,r2_pp]
}
df_result_prophet = pd.DataFrame(data = dados_prophet['Resultado'], index=dados_prophet['Indicador'], columns =['Resultado'])
df_result_prophet

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt5nedm02/kf3j02xn.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt5nedm02/lmfgp2v5.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=93644', 'data', 'file=/tmp/tmpt5nedm02/kf3j02xn.json', 'init=/tmp/tmpt5nedm02/lmfgp2v5.json', 'output', 'file=/tmp/tmpt5nedm02/prophet_modelz_73t11v/prophet_model-20240515210425.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:04:25 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
21:04:26 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


,Resultado
Erro Médio Absoluto - MAE,5.96
Erro Quadrático Médio - MSE,43.35
Raiz Quadrada do Erro Médio - RMSE,6.58
Média Percentual Absoluta do Erro - MAPE,0.07
Coeficiente de Determinação(R²),-1.14


MODELO LSTM

Vamos agora utilizar uma Rede LSTM (Long Short-Term Memory)


In [17]:
df_LSTM = pd.DataFrame({indice: dados_acao['Close']})
df_LSTM.reset_index(inplace=True)

#Aplicando suavização exponencial
alpha = 0.15   # Fator de suavização

# O parâmetro alpha na suavização exponencial controla a taxa de decaimento dos pesos atribuídos às observações passadas.
# Determina o quão rapidamente o impacto das observações antigas diminui à medida que você avança no tempo.

df_LSTM['Smoothed_Close'] = df_LSTM[indice].ewm(alpha=alpha, adjust=False).mean()
close_data = df_LSTM[indice].values #fechamento não suavizado
close_data = close_data.reshape(-1,1) #transformar em array

#Agora aplicamos a normalização dos dados para não termos ruído
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(close_data)
close_data_escalado = scaler.transform(close_data)

#Separando as bases em treino e teste
look_back = 10
close_train_lstm = close_data_escalado[:-steps]
close_test_lstm = close_data_escalado[-(steps):]

date_train_lstm = df_LSTM['Date'][:-steps]
date_test_lstm = df_LSTM['Date'][-steps:]

# Gerar sequências temporais para treinamento e teste em um modelo de aprendizado de máquina
train_generator = TimeseriesGenerator(close_train_lstm, close_train_lstm, length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(close_test_lstm, close_test_lstm, length=look_back, batch_size=1)

#Aplica o modelo
np.random.seed(7)
model_lstm = Sequential()
model_lstm.add(LSTM(100, activation='relu', input_shape=(look_back,1)))
model_lstm.add(Dense(1)),
model_lstm.compile(optimizer='adam', loss='mean_squared_error')
num_epochs = 20
retornomodelo = model_lstm.fit(train_generator, epochs=num_epochs, verbose=1)

# 1. Fazer previsões usando o conjunto de teste
test_predictions_lstm = model_lstm.predict(test_generator)
prediction_lstm = test_predictions_lstm.reshape((-1))
close_data_g = close_data_escalado.reshape((-1))

# Plota um gráfico para comparar o realizado com o periodo testado + previsões
df1 = pd.DataFrame(data= close_data_g, index=df_LSTM['Date'].values, columns=['Dados Históricos'] )
#df1.rename(columns={indice: 'Dados Históricos'}, inplace = True)
df2 = pd.DataFrame(data= prediction_lstm, index=date_test_lstm[-prediction_lstm.size:], columns=['Predições'] )
#df2.rename(columns={indice: 'Predições'}, inplace = True)
df3 = pd.DataFrame(data=[])
df4 = pd.DataFrame(data=[])

figLSTM = cria_grafico(df1, df2,df3,df4,'Predições com Modelo LSTM', 1,steps+1 )
fig.show()

#calcula as métricas de avaliação de desempenho do modelo
MAE_LSTM = mean_absolute_error(close_data_g[-prediction_lstm.size:], prediction_lstm)
MSE_LSTM = mean_squared_error(close_data_g[-prediction_lstm.size:], prediction_lstm, squared=True)
RMSE_LSTM = mean_squared_error(close_data_g[-prediction_lstm.size:], prediction_lstm, squared=False)
MAPE_LSTM = mean_absolute_percentage_error(close_data_g[-prediction_lstm.size:], prediction_lstm)
r2_LSTM = r2_score(close_data_g[-prediction_lstm.size:], prediction_lstm)
dados_LSTM = {
'Indicador': Lista_indicadores,
'Resultado': [MAE_LSTM, MSE_LSTM, RMSE_LSTM,MAPE_LSTM,r2_LSTM]
}
df_result_LSTM = pd.DataFrame(data = dados_LSTM['Resultado'], index=dados_LSTM['Indicador'], columns =['Resultado'])
df_result_LSTM

predictions_LSTM_inv = scaler.inverse_transform(prediction_lstm.reshape(-1, 1))
df_LSTM_pred_g = pd.DataFrame(data=predictions_LSTM_inv, columns = [indice], index= df_LSTM[-predictions_LSTM_inv.size:]['Date'])

Epoch 1/20
185/185 [==============================] - 3s 8ms/step - loss: 0.0249
Epoch 2/20
185/185 [==============================] - 1s 8ms/step - loss: 9.0235e-04
Epoch 3/20
185/185 [==============================] - 2s 9ms/step - loss: 7.4666e-04
Epoch 4/20
185/185 [==============================] - 2s 12ms/step - loss: 8.3632e-04
Epoch 5/20
185/185 [==============================] - 2s 13ms/step - loss: 6.6370e-04
Epoch 6/20
185/185 [==============================] - 2s 8ms/step - loss: 7.4141e-04
Epoch 7/20
185/185 [==============================] - 2s 13ms/step - loss: 6.8205e-04
Epoch 8/20
185/185 [==============================] - 2s 8ms/step - loss: 7.0877e-04
Epoch 9/20
185/185 [==============================] - 2s 8ms/step - loss: 6.1720e-04
Epoch 10/20
185/185 [==============================] - 2s 9ms/step - loss: 6.6549e-04
Epoch 11/20
185/185 [==============================] - 2s 12ms/step - loss: 5.3146e-04
Epoch 12/20
185/185 [==============================] - 2s 8ms/s

MODELO LSTM SUAVIZADO

Vamos agora utilizar uma Rede LSTM Suavizada (Long Short-Term Memory)

In [18]:
cs_data =  df_LSTM['Smoothed_Close'].values #fechamento  suavizado
cs_data = cs_data.reshape(-1,1) #transformar em array
cs_data_escalado = scaler.transform(cs_data)

#Separando as bases em treino e teste
cs_train_lstm = cs_data_escalado[:-steps]
cs_test_lstm = cs_data_escalado[-steps:]

# Gerar sequências temporais para treinamento e teste em um modelo de aprendizado de máquina
look_back_cs = 5
train_generator_cs = TimeseriesGenerator(cs_train_lstm, cs_train_lstm, length=look_back_cs, batch_size=20)
test_generator_cs = TimeseriesGenerator(cs_test_lstm, cs_test_lstm, length=look_back_cs, batch_size=1)

#Aplica o modelo
np.random.seed(7)
model_cs = Sequential()
model_cs.add(LSTM(100, activation='relu', input_shape=(look_back_cs,1)))
model_cs.add(Dense(1)),
model_cs.compile(optimizer='adam', loss='mean_squared_error')
num_epochs = 20
model_cs.fit(train_generator_cs, epochs=num_epochs, verbose=1)

# 1. Fazer previsões usando o conjunto de teste
test_predictions_cs_lstm = model_cs.predict(test_generator_cs)
prediction_lstm_cs = test_predictions_cs_lstm.reshape((-1))

# Plota um gráfico para comparar o realizado com o periodo testado + previsões
df1 = pd.DataFrame(data= close_data_g, index=df_LSTM['Date'].values, columns=['Dados Históricos'] )
df1.rename(columns={indice: 'Dados Históricos'}, inplace = True)
df2 = pd.DataFrame(data= prediction_lstm_cs, index=date_test_lstm[-prediction_lstm_cs.size:], columns=['Predições'] )
df2.rename(columns={indice: 'Predições'}, inplace = True)
df3 = pd.DataFrame(data=[])
df4 = pd.DataFrame(data=[])

figLSTMCS = cria_grafico(df1, df2,df3,df4,'Predições com Modelo LSTM - Curva Suavizada', 1,steps+1 )
fig.show()

#calcula as métricas de avaliação de desempenho do modelo
MAE_CS = mean_absolute_error(close_data_g[-prediction_lstm_cs.size:], prediction_lstm_cs)
MSE_CS = mean_squared_error(close_data_g[-prediction_lstm_cs.size:], prediction_lstm_cs, squared=True)
RMSE_CS = mean_squared_error(close_data_g[-prediction_lstm_cs.size:], prediction_lstm_cs, squared=False)
MAPE_CS = mean_absolute_percentage_error(close_data_g[-prediction_lstm_cs.size:], prediction_lstm_cs)
r2_CS = r2_score(close_data_g[-prediction_lstm_cs.size:], prediction_lstm_cs)

dados_LSTM_CS = {
'Indicador': Lista_indicadores,
'Resultado': [MAE_CS, MSE_CS,RMSE_LSTM,MAPE_CS,r2_CS]
}
df_result_LSTM_CS = pd.DataFrame(data = dados_LSTM_CS['Resultado'], index=dados_LSTM_CS['Indicador'], columns =['Resultado'])
df_result_LSTM_CS

predictions_LSTM_cs_inv = scaler.inverse_transform(prediction_lstm_cs.reshape(-1, 1))
df_LSTM_pred_cs_g = pd.DataFrame(data=predictions_LSTM_cs_inv, columns = [indice], index= df_LSTM[-predictions_LSTM_cs_inv.size:]['Date'])

Epoch 1/20
185/185 [==============================] - 3s 6ms/step - loss: 0.0420
Epoch 2/20
185/185 [==============================] - 1s 6ms/step - loss: 2.8326e-04
Epoch 3/20
185/185 [==============================] - 2s 8ms/step - loss: 1.7899e-04
Epoch 4/20
185/185 [==============================] - 1s 8ms/step - loss: 2.6533e-04
Epoch 5/20
185/185 [==============================] - 1s 6ms/step - loss: 2.6800e-04
Epoch 6/20
185/185 [==============================] - 1s 6ms/step - loss: 2.7307e-04
Epoch 7/20
185/185 [==============================] - 1s 6ms/step - loss: 2.0304e-04
Epoch 8/20
185/185 [==============================] - 1s 6ms/step - loss: 2.3758e-04
Epoch 9/20
185/185 [==============================] - 1s 6ms/step - loss: 2.4724e-04
Epoch 10/20
185/185 [==============================] - 1s 6ms/step - loss: 2.1119e-04
Epoch 11/20
185/185 [==============================] - 1s 6ms/step - loss: 2.3871e-04
Epoch 12/20
185/185 [==============================] - 2s 10ms/step

MODELO XGBoost

Vamos agora utilizar o modelo XGBoost para previsões.

In [19]:
df_XGB = pd.DataFrame({indice: dados_acao['Close']})
df_XGB.reset_index(inplace=True)

# Aplicando suavização exponencial
alpha = 0.15
df_XGB['Smoothed_Close'] = df_XGB[indice].ewm(alpha=alpha, adjust=False).mean()
XG_close_data = df_XGB['Smoothed_Close'].values.reshape(-1, 1)

# Normalizando os dados
XG_scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = XG_scaler.fit_transform(XG_close_data)

#Separando as bases em treino e teste
def create_dataset(X, look_back):
    dataX, dataY = [], []
    for i in range(len(X) - look_back):
        a = X[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(X[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 10
train_size = int(len(X_scaled) * 0.8)
X_train, X_test = X_scaled[:train_size], X_scaled[train_size - look_back:]

X_train, y_train = create_dataset(X_train, look_back)
X_test, y_test = create_dataset(X_test, look_back)

# Criando e treinando o modelo XGBoost
XG_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=7)
XG_model.fit(X_train, y_train)

# Fazendo previsões
y_pred = XG_model.predict(X_test)
y_pred_rescaled = XG_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
y_test_rescaled = XG_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

dates_for_predictions = df_XGB['Date'].values[-len(y_pred):]

# Plota um gráfico para comparar o realizado com o periodo testado + previsões
df1_xg = pd.DataFrame(data=XG_scaler.inverse_transform(X_scaled).flatten(), index=df_XGB['Date'], columns=['Dados Históricos'])
df2_xg = pd.DataFrame(data=y_pred_rescaled, index=dates_for_predictions, columns=['Predições'])

fig_XG = cria_grafico(df1_xg, df2_xg, pd.DataFrame(data=[]), pd.DataFrame(data=[]),'Predições com o algoritmo XGBoost', 1, steps+1)
fig.show()

# Calcula as métricas de avaliação de desempenho do modelo
MAE_XGB = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
MSE_XGB = mean_squared_error(y_test_rescaled, y_pred_rescaled)
RMSE_XGB = np.sqrt(MSE_XGB)
MAPE_XGB = mean_absolute_percentage_error(y_test_rescaled, y_pred_rescaled)
r2_XGB = r2_score(y_test_rescaled, y_pred_rescaled)
df_result_XGB = pd.DataFrame(data=[MAE_XGB, MSE_XGB, RMSE_XGB, MAPE_XGB, r2_XGB], index=['MAE', 'MSE', 'RMSE', 'MAPE', 'R2'], columns=['Resultado'])
df_result_XGB

,Resultado
MAE,0.46
MSE,0.39
RMSE,0.63
MAPE,0.01
R2,1.00


Resumo dos Resultados:

Agora que já testamos diversos modelos clássicos de sérias temporais, como escolher um? Para medir um modelo (seja ele qual for), vamos tentar medir e analisar os erros que ele apresenta, ou seja, vamos comparar Y e Ŷ (Y real e Y previsto, respectivamente) e dar atenção à esses resíduos. Sendo assim, a seguir, vamos comparar 5 técnicas diferentes para aferir o melhor modelo de previsão para séries temporais:



In [20]:
dados_consolidados = {
    'Indicador': Lista_indicadores,
    'ARIMA': [MAE_ARIMA, MSE_ARIMA,RMSE_ARIMA,MAPE_ARIMA,r2_ARIMA],
    'PROPHET': [MAE_pp, MSE_pp,RMSE_pp,MAPE_pp,r2_pp],
    'LSTM': [MAE_LSTM, MSE_LSTM,RMSE_LSTM,MAPE_LSTM,r2_LSTM],
    'LSTM Curva Suavizada': [MAE_CS, MSE_CS,RMSE_LSTM,MAPE_CS,r2_CS],
    'XGBoost': [MAE_XGB, MSE_XGB, RMSE_XGB, MAPE_XGB, r2_XGB]
    }

df_result_consolidado = pd.DataFrame(data = dados_consolidados, index=dados_consolidados['Indicador'], columns =['ARIMA','PROPHET','LSTM','LSTM Curva Suavizada', 'XGBoost'])
df_result_consolidado

,ARIMA,PROPHET,LSTM,LSTM Curva Suavizada,XGBoost
Erro Médio Absoluto - MAE,0.01,5.96,0.02,0.03,0.46
Erro Quadrático Médio - MSE,0.00,43.35,0.00,0.00,0.39
Raiz Quadrada do Erro Médio - RMSE,0.02,6.58,0.02,0.02,0.63
Média Percentual Absoluta do Erro - MAPE,0.00,0.07,0.03,0.05,0.01
Coeficiente de Determinação(R²),0.91,-1.14,0.81,0.37,1.00


Comparando a tabela acima, tanto o ARIMA quanto o LSTM alcaçaram bons resultados. Vamos compará-los graficamente para não restar dúvidas:

In [21]:
# Plota um gráfico para comparar todos os modelos
df1 = pd.DataFrame(df_cotacoes)
df1.rename(columns={indice: 'Dados Históricos'}, inplace = True)
df2 = pd.DataFrame(diff_df_inverted)
df2.rename(columns={indice: 'ARIMA'}, inplace = True)
df3 = pd.DataFrame(df_LSTM_pred_g)
df3.rename(columns={indice: 'LSTM'}, inplace = True)
df4 = pd.DataFrame(data=[])

figConsolidado = cria_grafico(df1, df2,df3,df4,'Comparativo de Predições dos Modelos - ARIMA vs LSTM', 1,steps+1 )
figConsolidado.show()

Na análise visual, o modelo ARIMA acompanhou melhor o comportamento do Índice BRENT, acertando o movimento (valorização ou desvalorização) enquanto o modelo LSTM ficou mais perto dos valores, porém com os valores com tendencias de subida, sem acompanhar o movimento diário. Vamos ver na prática como foram estes resultados pelo percentual de acertos do movimento:


In [22]:
#Monta um DF para armazenar os dados de previsão das ações
x = 0
prev_ini = date.today() + timedelta(days = 1)
na = [prev_ini]

while (len(na)+1) <= du :
  prev_fim_d = prev_ini + timedelta(days = x+1)
  x=x+1
  if (prev_fim_d.weekday() not in (5,6)):
    prev_fim = prev_fim_d
    na.append(prev_fim)

df_dt_futura = pd.DataFrame({"Date":na})

#Cria um segundo DF para unir as cotações correntes e as previsões das ações
df_cotacao_futura = pd.DataFrame({"Date":df_cotacoes.index.values})
df_cotacao_futura = pd.concat([df_cotacao_futura, df_dt_futura])
df_cotacao_futura['Date'] = pd.to_datetime(df_cotacao_futura['Date'])

#Monta os dataframes. A ideia é testar com os últimos <steps> dias e treinar com os dias anteriores
df_treina = df_cotacoes[:-steps]
df_teste  = df_cotacoes[-steps:]
df_prev   = df_cotacao_futura[-(steps+du):]
df_teste_g = pd.DataFrame(df_teste.index)
df_teste_g["teste"] = df_teste[indice].values


In [23]:
#ARIMA
df_compara_arima = pd.DataFrame(data=df_teste_arima[indice].values, index=df_teste_arima.index.values, columns=["Close"])
df_compara_arima["predicoes"] = df_forecast[indice].values
comp_real_arima = np.where(df_compara_arima['Close'].shift(1) < df_compara_arima['Close'], 1, 0)
comp_predito_arima = np.where(df_compara_arima['predicoes'].shift(1) < df_compara_arima['predicoes'], 1, 0)

df_compara_arima["comportamento_real"]= comp_real_arima
df_compara_arima["comportamento_predito"]= comp_predito_arima
acertou_arima = np.where(df_compara_arima['comportamento_real'] == df_compara_arima['comportamento_predito'], 1, 0)
df_compara_arima['acertou_o_lado'] = acertou_arima

#calcular media de acertos
df_compara_arima['var_percent_acao'] = df_compara_arima['Close'].pct_change()
df_compara_arima['var_percent_modelo'] = df_compara_arima['predicoes'].pct_change()
df_compara_arima['dif_var_percent']= (df_compara_arima['var_percent_acao'] - df_compara_arima['var_percent_modelo'])

acertou_lado_arima = df_compara_arima['acertou_o_lado'].sum()/len(df_compara_arima['acertou_o_lado'])
acertos_arima = acertou_lado_arima * 100


In [24]:
#LSTM
df_compara_LSTM = pd.DataFrame(data=df_teste_g[-df_LSTM_pred_g[indice].values.size:]["teste"].values, index=df_LSTM_pred_g.index, columns=["Close"])
df_compara_LSTM["predicoes"] = df_LSTM_pred_g[indice].values

comp_real_LSTM = np.where(df_compara_LSTM['Close'].shift(1) < df_compara_LSTM['Close'], 1, 0)
comp_predito_LSTM = np.where(df_compara_LSTM['predicoes'].shift(1) < df_compara_LSTM['predicoes'], 1, 0)

df_compara_LSTM["comportamento_real"]= comp_real_LSTM
df_compara_LSTM["comportamento_predito"]= comp_predito_LSTM
acertou_LSTM = np.where(df_compara_LSTM['comportamento_real'] == df_compara_LSTM['comportamento_predito'], 1, 0)
df_compara_LSTM['acertou_o_lado'] = acertou_LSTM

#calcular media de acertos
df_compara_LSTM['var_percent_acao'] = df_compara_LSTM['Close'].pct_change()
df_compara_LSTM['var_percent_modelo'] = df_compara_LSTM['predicoes'].pct_change()
df_compara_LSTM['dif_var_percent']= (df_compara_LSTM['var_percent_acao'] - df_compara_LSTM['var_percent_modelo'])

acertou_lado_LSTM = df_compara_LSTM['acertou_o_lado'].sum()/len(df_compara_LSTM['acertou_o_lado'])
acertos_LSTM = acertou_lado_LSTM * 100



In [25]:
dados_compararativos = {
    'Modelo': ['ARIMA','LSTM'],
    '% Acertos': [acertos_arima.round(2),acertos_LSTM.round(2)]
    }

df_comparativo = pd.DataFrame(data = dados_compararativos, index=dados_compararativos['Modelo'], columns =['% Acertos'])
df_comparativo

,% Acertos
ARIMA,40.83
LSTM,44.55


Como LTSM performou ligeiramente melhor, iremos optar por esse modelo

In [27]:
#Gera as previsões
close_prev_lstm = close_data_escalado[-(steps+du+look_back):]
data_prev_lstm = df_cotacao_futura[-(steps+du):]
prev_generator = TimeseriesGenerator(close_prev_lstm, close_prev_lstm, length=look_back, batch_size=1)
previsions_lstm = model_lstm.predict(prev_generator)
prev_lstm = previsions_lstm.reshape((-1))


prev_lstm_inv = scaler.inverse_transform(prev_lstm.reshape(-1, 1))
df_LSTM_prev_g = pd.DataFrame(data=prev_lstm_inv, columns = [indice], index= df_cotacao_futura[-prev_lstm_inv.size:]['Date'])

df = df_dt_futura['Date'][-1:].values
di = df_cotacoes.index[-(steps+du+look_back):]

layoutPREV= go.Layout(
                  title = 'Previsões - Petróleo Brent - Próximos 10 dias',
                  titlefont = dict(size=20))

figPrev = go.Figure(layout=layoutPREV)
figPrev.add_trace(go.Scatter(x=df_cotacoes.index.values, y=df_cotacoes[indice].values,
                      mode='lines',
                      name='Dados Históricos'))

figPrev.add_trace(go.Scatter(x=df_LSTM_pred_g.index.values, y=df_LSTM_pred_g[indice].values,
                    mode='lines',
                    name='Predição'))
figPrev.add_trace(go.Scatter(x=df_dt_futura['Date'] , y=df_LSTM_prev_g[indice].values,
                      mode='lines',
                     name='Previsão'))



figPrev.update_xaxes(
      rangeslider_visible=True,
      rangeselector=dict(
          buttons=list([
              dict(count=1, label="1m", step="month", stepmode="backward"),
              dict(count=6, label="6m", step="month", stepmode="backward"),
              dict(count=1, label="YTD", step="year", stepmode="todate"),
              dict(count=1, label="1y", step="year", stepmode="backward"),
              dict(step="all")
          ])
      ),
      range=(di[0].strftime('%Y-%m-%d'), df[0].strftime('%Y-%m-%d'))
  )

figPrev.show()


130/130 [==============================] - 1s 9ms/step


### Deploy em Produção

Para a aplicação em produção, não é necessário rodar fazer essa análise todas as vezes. Escolhe-se um modelo, que nesse caso será o LTSM, salva o modelo e carrega na aplicação

In [28]:
# Importações

import pickle as pkl

Salvar o modelo

In [29]:
with open('modelo_lstm.pkl', 'wb') as file:
  pkl.dump(model_lstm, file)


Carregando o modelo

In [ ]:
with open('modelo_ltsm.pkl', 'rb') as file_2:
  modelo_ltsm = pkl.load(file_2)